# **Question Answering❓**
with fine-tuned BERT on SQuAD 2.0.  

Question answering comes in many forms. We’ll look at the particular type of extractive QA that involves answering a question about a passage by highlighting the segment of the passage that answers the question. This involves fine-tuning a model which predicts a start position and an end position in the passage. More specifically, we will fine tune the [bert-base-uncased](https://huggingface.co/bert-base-uncased) model on the [SQuAD 2.0](https://rajpurkar.github.io/SQuAD-explorer/) dataset.

I have followed [this tutorial](https://huggingface.co/transformers/v3.2.0/custom_datasets.html#question-answering-with-squad-2-0) from the huggingface community for how to fine tune BERT on custom datasets which in our case is the SQuAD 2.0.

**Some first imports**

In [2]:
import requests
import json
import torch
import os
from tqdm import tqdm
import random

# from google.colab import drive
# drive.mount('/content/drive')

**Connecting Google Drive in order to save the model**

In [ ]:
# if not os.path.exists('/content/drive/MyDrive/BERT-SQuAD'):
#   os.mkdir('/content/drive/MyDrive/BERT-SQuAD')

In [3]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


### **Download SQuAD 2.0 ⬇️**

SQuAD consists of two json files.

* train dataset 
* validation dataset

In [4]:
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2023-03-14 18:32:43--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   253MB/s    in 0.2s    

2023-03-14 18:32:43 (253 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2023-03-14 18:32:43--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

## **Data preprocessing 💽**

In short, we have to do the following:

1. Extract the data from the jsons files
2. Tokenize the data
3. Define the datasets

### **Get data 📁** 

After we got a taste of the jsons files data format let's extract our data and store them into some data structures.

In [4]:
def read_data(path):  
  # load the json file
  with open(path, 'rb') as f:
    squad = json.load(f)

  contexts = []
  questions = []
  answers = []

  for group in squad['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts, questions, answers

random.seed(42)
def split_dataset(contexts, questions, answers):
  num_elements = int(len(contexts) * 0.1)
  sample_indices = random.sample(range(len(contexts)), num_elements)

  contexts_sample = [contexts[i] for i in sample_indices]
  questions_sample = [questions[i] for i in sample_indices]
  answers_sample = [answers[i] for i in sample_indices]
  
  return contexts_sample, questions_sample, answers_sample

Put the contexts, questions and answers for training and validation into the appropriate lists.

In [6]:
train_contextl, train_questionl, train_answerl = read_data('squad/train-v2.0.json')
train_contexts, train_questions, train_answers = split_dataset(train_contextl, train_questionl, train_answerl)
valid_contexts, valid_questions, valid_answers = read_data('squad/dev-v2.0.json')
print(len(train_contexts), len(valid_contexts))

8682 20302


As you can see above, the answers are dictionaries whith the answer text and an integer which indicates the start index of the answer in the context. As the SQuAD does not give us the end index of the answer in the context we have to find it ourselves. So, let's get the character position at which the answer ends in the passage. Note that sometimes SQuAD answers are off by one or two characters, so we will also adjust for that.

In [7]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(valid_answers, valid_contexts)
# You can see that now we get the answer_end also
print(train_questions[-1000])
print(train_answers[-1000])

The often unclear division between a self-executing treaty and a non-self-executing treaty can lead to a treaty being what if disagreements exist within a party?
{'text': 'politicized', 'answer_start': 61, 'answer_end': 72}


### **Tokenization 🔢**

As we know we have to tokenize our data in form that is acceptable for the BERT model. We are going to use the `BertTokenizerFast` instead of `BertTokenizer` as the first one is much faster. Since we are going to train our model in batches we need to set `padding=True`.

In [43]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
model_name = "bert-base-cased"
# Load a pre-trained checkpoint
checkpoint_path = 'squad/checkpoint-57950'

# load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# # Load the checkpoint into the model
# pre_trained_weight = torch.load(os.path.join(checkpoint_path, "pytorch_model.bin"))

# # pre_trained_weight.pop('bert.pooler.dense.weight')
# # pre_trained_weight.pop('bert.pooler.dense.bias')
# weight = pre_trained_weight.pop('classifier.weight')
# bias = pre_trained_weight.pop('classifier.bias')

# pre_trained_weight['qa_outputs.weight'] = weight[:2]
# pre_trained_weight['qa_outputs.bias'] = bias[:2]

# model.load_state_dict(pre_trained_weight)

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

Next we need to convert our character start/end positions to token start/end positions. Why is that? Because our words converted into tokens, so the answer start/end needs to show the index of start/end token which contains the answer and not the specific characters in the context.

In [44]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length

    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(valid_encodings, valid_answers)

### **Dataset definition 🗄️**

We have to define our dataset using the PyTorch Dataset class from `torch.utils` in order create our dataloaders after that.

In [45]:
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

train_dataset = SQuAD_Dataset(train_encodings)
valid_dataset = SQuAD_Dataset(valid_encodings)

### **Dataloaders 🔁**

In [49]:
from torch.utils.data import DataLoader

batch_size = 32

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

## **Fine-Tuning ⚙️**

### **Model definition 🤖**

We are going to use the `bert-case-uncased` from the huggingface transformers.

### **Training 🏋️‍♂️**

Μy choices for some parameters:

* Use of `AdamW` which is a stochastic optimization method that modifies the typical implementation of weight decay in Adam, by decoupling weight decay from the gradient update. This helps to avoid overfitting which is necessary in this case were the model is very complex.


In [50]:
# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [48]:
from transformers import AdamW

# Freeze the backbone parameters
# for param in model.bert.parameters():
#     param.requires_grad = False

N_EPOCHS = 5

optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)

model.train()
train_losses = []


for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  loss_of_epoch = 0

  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()
    loss_of_epoch += loss.item()
    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

  loss_of_epoch /= len(train_loader)
  train_losses.append(loss_of_epoch)


/home/bowen/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/136 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 7.79 GiB total capacity; 6.81 GiB already allocated; 57.62 MiB free; 6.96 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [37]:
print(train_losses)

[8.546129494905472, 6.665331488146501, 5.701449467855341, 5.44834376783932, 5.330090836567037]


**Save the model in my drive in order not to run it each time**

In [22]:
# from google.colab import drive
# drive.mount('/content/drive')

# model_path = '/content/drive/MyDrive/BERT-SQuAD/Model_weights_HW1/NER/MRC_11590'
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/BERT-SQuAD/MRC_finetuen/tokenizer_config.json',
 '/content/drive/MyDrive/BERT-SQuAD/MRC_finetuen/special_tokens_map.json',
 '/content/drive/MyDrive/BERT-SQuAD/MRC_finetuen/vocab.txt',
 '/content/drive/MyDrive/BERT-SQuAD/MRC_finetuen/added_tokens.json',
 '/content/drive/MyDrive/BERT-SQuAD/MRC_finetuen/tokenizer.json')

**Respectively, load the saved model**

In [ ]:
#from transformers import BertForQuestionAnswering, BertTokenizerFast

#model_path = '/content/drive/MyDrive/BERT-SQuAD'
#model = BertForQuestionAnswering.from_pretrained(model_path)
#tokenizer = BertTokenizerFast.from_pretrained(model_path)

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#print(f'Working on {device}')

#model = model.to(device)

### **Testing ✅**

We are evaluating the model on the validation set by checking the model's predictions for the answer's start and end indexes and comparing with the true ones.

In [ ]:
# model.eval()

# acc = []


# for batch in tqdm(valid_loader):

#   with torch.no_grad():
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     start_true = batch['start_positions'].to(device)
#     end_true = batch['end_positions'].to(device)
    
#     outputs = model(input_ids, attention_mask=attention_mask)

#     start_pred = torch.argmax(outputs['start_logits'], dim=1)
#     end_pred = torch.argmax(outputs['end_logits'], dim=1)

#     acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
#     acc.append(((end_pred == end_true).sum()/len(end_pred)).item())




In [ ]:
# print(acc)
# accuracy = sum(acc)/len(acc)
# print("Validation Accuracy is {}".format(accuracy))

# print("\n\nT/P\tanswer_start\tanswer_end\n")
# for i in range(len(start_true)):
#   print(f"true\t{start_true[i]}\t{end_true[i]}\n"
#         f"pred\t{start_pred[i]}\t{end_pred[i]}\n")

In [38]:
model.eval()

valid_predictions = []
val_answers = []
acc = []
for i in tqdm(range(len(valid_contexts))):
    encoded_dict = tokenizer.encode_plus(valid_questions[i], valid_contexts[i], return_tensors='pt', max_length=512, truncation=True)
    input_ids = encoded_dict['input_ids'].to(device)
    token_type_ids = encoded_dict['token_type_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        start_logits, end_logits = outputs.start_logits, outputs.end_logits

    start_index = torch.argmax(start_logits, dim=1)[0]
    end_index = torch.argmax(end_logits, dim=1)[0]
    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)
    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
    valid_predictions.append(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded_dict['input_ids'][0][start_index:end_index+1])))
    val_answers.append(valid_answers[i]['text'])

accuracy = sum(acc)/len(acc)
print("Validation Accuracy is {}".format(accuracy))

100%|██████████| 20302/20302 [05:10<00:00, 65.29it/s]

Validation Accuracy is 0.06334351295438873


In [23]:
def get_prediction(context, question):
  inputs = tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
  outputs = model(**inputs)
  
  answer_start = torch.argmax(outputs[0])  
  answer_end = torch.argmax(outputs[1]) + 1 
  
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
  
  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re
  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)
  def white_space_fix(text):
    return " ".join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match(prediction, truth):
    return bool(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  
  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  common_tokens = set(pred_tokens) & set(truth_tokens)
  
  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0
  
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  F1 = round(2 * (prec * rec) / (prec + rec), 2)
  # print(prec, rec, F1)
  return F1
  
def question_answer(context, question, answer):
  prediction = get_prediction(context,question)
  em_score = exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f'Question: {question}')
  print(f'Prediction: {prediction}')
  print(f'True Answer: {answer}')
  print(f'Exact match: {em_score}')
  print(f'F1 score: {f1_score}\n')

In [24]:
EM = []
F1 = []
Prec = []
Rec = []
for pre, val in zip(valid_predictions, val_answers):
  em_score = exact_match(pre, val)
  f1_score = compute_f1(pre, val)
  EM.append(em_score)
  F1.append(f1_score)
  # Prec.append(prec)
  # Rec.append(rec)

print(f'Validate number: {len(EM)}')
print(f'Exact match number: {sum(EM)}')
print(f'Exact match: {sum(EM)/len(EM)}')
# print(f'Precision:   {sum(EM)/len(EM)}')
# print(f'Recall:      {sum(EM)/len(EM)}')
print(f'F1 score:    {sum(F1)/len(F1)}')


Validate number: 20302
Exact match number: 279
Exact match: 0.013742488424785735
F1 score:    0.04945473352378949


### **Ask questions 🙋**

We are going to use some functions from the [*official Evaluation Script v2.0*](https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/) of SQuAD in order to test the fine-tuned model by asking some questions given a context. I have also looked at this [notebook](https://colab.research.google.com/github/fastforwardlabs/ff14_blog/blob/master/_notebooks/2020-06-09-Evaluating_BERT_on_SQuAD.ipynb#scrollTo=MzPlHgWEBQ8D) which evaluates BERT on SQuAD.

**Beyoncé**

In [39]:
context = """Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, 
          songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing 
          and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. 
          Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. 
          Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, 
          earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy"."""


questions = ["For whom the passage is talking about?",
             "When did Beyonce born?",
             "Where did Beyonce born?",
             "What is Beyonce's nationality?",
             "Who was the Destiny's group manager?",
             "What name has the Beyoncé's debut album?",
             "How many Grammy Awards did Beyonce earn?",
             "When did the Beyoncé's debut album release?",
             "Who was the lead singer of R&B girl-group Destiny's Child?"]

answers = ["Beyonce Giselle Knowles - Carter", "September 4, 1981", "Houston, Texas", 
           "American", "Mathew Knowles", "Dangerously in Love", "five", "2003", 
           "Beyonce Giselle Knowles - Carter"]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

Question: For whom the passage is talking about?
Prediction: 
True Answer: Beyonce Giselle Knowles - Carter
Exact match: False
F1 score: 0

Question: When did Beyonce born?
Prediction: 
True Answer: September 4, 1981
Exact match: False
F1 score: 0

Question: Where did Beyonce born?
Prediction: 
True Answer: Houston, Texas
Exact match: False
F1 score: 0

Question: What is Beyonce's nationality?
Prediction: 
True Answer: American
Exact match: False
F1 score: 0

Question: Who was the Destiny's group manager?
Prediction: 
True Answer: Mathew Knowles
Exact match: False
F1 score: 0

Question: What name has the Beyoncé's debut album?
Prediction: 
True Answer: Dangerously in Love
Exact match: False
F1 score: 0

Question: How many Grammy Awards did Beyonce earn?
Prediction: 
True Answer: five
Exact match: False
F1 score: 0

Question: When did the Beyoncé's debut album release?
Prediction: 
True Answer: 2003
Exact match: False
F1 score: 0

Question: Who was the lead singer of R&B girl-group Dest

**Athens**

In [40]:
context = """Athens is the capital and largest city of Greece. Athens dominates the Attica region and is one of the world's oldest cities, 
             with its recorded history spanning over 3,400 years and its earliest human presence starting somewhere between the 11th and 7th millennium BC.
             Classical Athens was a powerful city-state. It was a center for the arts, learning and philosophy, and the home of Plato's Academy and Aristotle's Lyceum.
             It is widely referred to as the cradle of Western civilization and the birthplace of democracy, largely because of its cultural and political impact on the European continent—particularly Ancient Rome.
             In modern times, Athens is a large cosmopolitan metropolis and central to economic, financial, industrial, maritime, political and cultural life in Greece. 
             In 2021, Athens' urban area hosted more than three and a half million people, which is around 35% of the entire population of Greece.
             Athens is a Beta global city according to the Globalization and World Cities Research Network, and is one of the biggest economic centers in Southeastern Europe. 
             It also has a large financial sector, and its port Piraeus is both the largest passenger port in Europe, and the second largest in the world."""

questions = ["Which is the largest city in Greece?",
             "For what was the Athens center?",
             "Which city was the home of Plato's Academy?"]

answers = ["Athens", "center for the arts, learning and philosophy", "Athens"]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

Question: Which is the largest city in Greece?
Prediction: Plato ' s Academy
True Answer: Athens
Exact match: False
F1 score: 0

Question: For what was the Athens center?
Prediction: Classical Athens was a powerful city - state. It was a center for the arts, learning and philosophy, and the home of Plato ' s Academy
True Answer: center for the arts, learning and philosophy
Exact match: False
F1 score: 0.46

Question: Which city was the home of Plato's Academy?
Prediction: Plato ' s Academy
True Answer: Athens
Exact match: False
F1 score: 0



**Angelos**

In [41]:
context = """Angelos Poulis was born on 8 April 2001 in Nicosia, Cyprus. He is half Cypriot and half Greek. 
            He is currently studying at the Department of Informatics and Telecommunications of the University of Athens in Greece. 
            His scientific interests are in the broad field of Artificial Intelligence and he loves to train neural networks! 
            Okay, I'm Angelos and I'll stop talking about me right now."""

questions = ["When did Angelos born?",
             "In what university is Angelos studying now?",
             "What is Angelos' nationality?",
             "What are his scientific interests?",
             "What I will do right now?"]

answers = ["8 April 2001", "University of Athens", 
           "half Cypriot and half Greek", "Artificial Intelligence", 
           "stop talking about me"]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

Question: When did Angelos born?
Prediction: Angelos born? [SEP] Angelos Pouli
True Answer: 8 April 2001
Exact match: False
F1 score: 0

Question: In what university is Angelos studying now?
Prediction: Angelos studying now? [SEP] Angelos Pouli
True Answer: University of Athens
Exact match: False
F1 score: 0

Question: What is Angelos' nationality?
Prediction: ##s Pouli
True Answer: half Cypriot and half Greek
Exact match: False
F1 score: 0

Question: What are his scientific interests?
Prediction: ##s Pouli
True Answer: Artificial Intelligence
Exact match: False
F1 score: 0

Question: What I will do right now?
Prediction: ##s Pouli
True Answer: stop talking about me
Exact match: False
F1 score: 0

